# Модель Transformer-2

Взять тот же датасет и предобученную модель для задачи суммаризации
1. Проверить насколько хорошо она суммаризирует
2. Сделать генерацию заголовков для статьи

In [1]:
# Загрузка данных

# !wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
# !wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
# !wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [2]:
# Функция чтения данных

import json
import random

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(json.loads(line))
    if sort_by_date:
        records.sort(key=lambda x: x["date"])
    if shuffle:
        random.shuffle
    return records

In [3]:
# train_records = read_gazeta_records("gazeta_train.txt")
# val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [4]:
import pandas as pd

dataset = pd.DataFrame(test_records)
dataset.head(3)

,url,text,title,summary,date
0,https://www.gazeta.ru/science/2020/02/14_a_129...,Американское аэрокосмическое агентство NASA ог...,"Венера, Ио или Тритон: куда полетит NASA","В NASA назвали четыре миссии в дальний космос,...",2020-02-14 16:39:11
1,https://www.gazeta.ru/social/2020/02/28/129806...,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,2020-02-28 10:44:13
2,https://www.gazeta.ru/politics/2019/11/07_a_12...,7 ноября в Белоруссии прошли выборы членов сов...,Вспомнить СССР: как Лукашенко провел выборы,В Белоруссии в день годовщины Октябрьской рево...,2019-11-07 19:55:08


In [5]:
dataset = dataset.drop(['url', 'date'], axis=1)

In [6]:
# Сделаем маленький тестовый датасет 

dataset = dataset[:20]

## Summarization

### Модель MBARTRuSumGazeta

In [7]:
# Загрузка модели 

from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [8]:
# Пример работы модели

article_text = dataset['text'][1]

input_ids = tokenizer(
    [article_text],
    max_length=600,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=4
)[0]

summary = tokenizer.decode(output_ids, skip_special_tokens=True)

print('Predicted summary: ', summary)
print('Real summary: ', dataset['summary'][1])

Predicted summary:  В Кремле прошла премьера новогоднего шоу «Танцуют все!» с участием более 300 артистов из одного региона. Зрителям рассказали, что в Бурятии сохранилась и развивается культура десятков национальностей, включая русских, бурятов, староверов (семейских), эвенков.
Real summary:  25 и 26 февраля в Кремлевском дворце съездов праздновали Сагаалган — Восточный Новый год. Бурятия - центр российского буддизма и один из немногих регионов страны, где новый год встречают официально дважды.


In [9]:
# Генерация краткого содержания для всего датасета

data = dataset['text']

summaries = []

for row in data:
    input_ids = tokenizer(
        [row],
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    summaries.append(summary)

In [10]:
dataset['pred_summary'] = summaries

In [11]:
dataset.sample(3)

,text,title,summary,pred_summary
3,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,Народная артистка РСФСР Надежда Бабкина в инте...,Народная артистка РСФСР Надежда Бабкина расска...
18,"Американскому круизному лайнеру Westerdam, на ...",«Нет больных? Откуда знаете?» Лайнер из США ни...,"Круизный лайнер Westerdam, принадлежащий амери...","Американскому круизному лайнеру Westerdam, на ..."
6,Выступление главы российской делегации Петра Т...,"«Смотрите, что у вас происходит»: как прервали...",Активист сорвал выступление главы делегации РФ...,Выступление главы российской делегации Петра Т...


In [12]:
# Загрузка библиотек для оценки качества 

from datasets import load_metric

rouge_score = load_metric("rouge")

In [18]:
predictions = dataset['pred_summary']
references = dataset['summary']

In [19]:
# Rouge

sum_scores = rouge_score.compute(
    predictions=[predictions], references=[references]
)
sum_scores

{'rouge1': AggregateScore(low=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), mid=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), high=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744)),
 'rouge2': AggregateScore(low=Score(precision=0.14634146341463414, recall=0.3157894736842105, fmeasure=0.19999999999999998), mid=Score(precision=0.14634146341463414, recall=0.3157894736842105, fmeasure=0.19999999999999998), high=Score(precision=0.14634146341463414, recall=0.3157894736842105, fmeasure=0.19999999999999998)),
 'rougeL': AggregateScore(low=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), mid=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744), high=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.41935483870967744)),
 'rougeLsum': AggregateScore(low=Score(precision=0.30952380952380953, recall=0.65, fmeasure=0.419354838709677

## Headlines

### Модель RuBertTelegramHeadlines

In [24]:
# Загрузка модели

from transformers import AutoTokenizer, EncoderDecoderModel

model_name = "IlyaGusev/rubert_telegram_headlines"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_case=False, do_basic_tokenize=False, strip_accents=False)
model = EncoderDecoderModel.from_pretrained(model_name)

In [25]:
# Пример работы модели

article_text = dataset['text'][1]

input_ids = tokenizer(
    [article_text],
    add_special_tokens=True,
    max_length=256,
    padding="max_length",
    truncation=True,
    return_tensors="pt",
)["input_ids"]

output_ids = model.generate(
    input_ids=input_ids,
    max_length=64,
    no_repeat_ngram_size=3,
    num_beams=10,
    top_p=0.95
)[0]

headline = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
print('Predicted headline: ', headline)
print('Real headline: ', dataset['title'][1])

Predicted headline:  В Кремле прошел самый лучший в России онлайн-концерт
Real headline:  «Люди в Бурятии очень талантливые»


In [26]:
# Генерация заголовков для всего датасета

data = dataset['text']

headlines = []

for row in data:
    input_ids = tokenizer(
        [row],
        add_special_tokens=True,
        max_length=256,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=64,
        no_repeat_ngram_size=3,
        num_beams=10,
        top_p=0.95
    )[0]
    
    headline = tokenizer.decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    headlines.append(headline)

In [27]:
dataset['pred_headline'] = headlines

In [28]:
dataset.sample(3)

,text,title,summary,pred_summary,pred_headline
1,Около 11 тысяч зрителей увидели все самое лучш...,«Люди в Бурятии очень талантливые»,25 и 26 февраля в Кремлевском дворце съездов п...,В Кремле прошла премьера новогоднего шоу «Танц...,В Кремле прошел самый лучший в России онлайн-к...
10,Спрос на бензин в России в 2019 году упал на 1...,Дорогое удовольствие: почему снизился спрос на...,Рекордное снижение спроса на бензин зафиксиров...,Спрос на бензин в России в 2019 году упал на 1...,«Известия»: Спрос на бензин в России упал на 1 %
3,Народная артистка РСФСР Надежда Бабкина в инте...,«Он очень переживал»: Бабкина об отношениях с ...,Народная артистка РСФСР Надежда Бабкина в инте...,Народная артистка РСФСР Надежда Бабкина расска...,Надежда Бабкина рассказала о романе с гражданс...


In [29]:
predictions = dataset['pred_headline']
references = dataset['title']

In [30]:
# Rouge

head_scores = rouge_score.compute(
    predictions=[predictions], references=[references]
)
head_scores

{'rouge1': AggregateScore(low=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), mid=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), high=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666)),
 'rouge2': AggregateScore(low=Score(precision=0.2222222222222222, recall=0.5, fmeasure=0.30769230769230765), mid=Score(precision=0.2222222222222222, recall=0.5, fmeasure=0.30769230769230765), high=Score(precision=0.2222222222222222, recall=0.5, fmeasure=0.30769230769230765)),
 'rougeL': AggregateScore(low=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), mid=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), high=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666)),
 'rougeLsum': AggregateScore(low=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), mid=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666), high=Score(precision=0.5, recall=1.0, fmeasure=0.6666666666666666))}